# Create and run a local RAG pipeline from scratch


## What is RAG ?

RAG stands for retrieval augmented Generation.

It was introduced in the paper [_Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks_](https://arxiv.org/abs/2005.11401).

The goal of RAG is to take information and pass it to an LLM so it can generate outputs based on that information.

- **Retrieval** --> Find Relevant information given a query , e.g. "what are the macronutrients and what do they do?" --> retrieves passages of the text related to the macronutrients from a nutrition textbook .

- **Augmented** --> To take the relevant information and augment out input(prompt) to an LLm with that relevant information

- **Generation**--> take result of above two steps and pass them on to a LLM for generative outputs


In [1]:
import torch

## Why RAG?

The main goal of RAG is to improve the generation outputs of LLMs .

1. To prevent hallucinations - LLMs are capable of generating _good looking_ texts , but that doesn't mean , it is factually correct , RAG can help LLMs to generate passage based on relevant passages that are factual .

2. Work with Custom Data - Many base LLMs are trained with internet-scale data. This means they have a fairly good understanding of language in general , However that also means the responses can be generic in nature , RAG helps generating based on specific data.


## What can be RAG used for?

- Customer Support QNA chat -- Treat your existing support docs as a resource and when a customer asks a question , you could have a retrieval system , retrieve relevant documentation snippets and then have an LLM craft those snippets into an answer .

- Email chain analysis -- Lets say you're a large insurance company and you have chains and chais of emails of customer claims . You have use a RAG pipeline to find revelant information from those emailand then use an LLM to process them into structured data.

- Company internal Documentation Chat

- TextBook Q&A -- Lets say you are a student and you've got a 1200 page textbook read textbook , you could build a RAG pipeline to go through and find relevant passages to the questions you have..

Common theme -- take your document to a query and process them with an LLM

From this angle , you can consider an LLM as a calculator for words.


## Why Local?

Fun...

Privacy , Speed and Cost

- Privacy -- IF you have a private documentation, maybe you dont want to send you information to an API , You want to setup an LLM and run it on your own Hardware.

- Speed -- Whenever you use an API , you have to send some kind of data across the internet which takes time. Running Locally means we dont have to wait for transfer of data

- Cost -- If You own you own hardware , the cost is paid , no or least operational cost , only Initial cost.

- no Vendor Lockin - if API shuts down , you dont have to worry


In [2]:
print(torch.backends.mps.is_available())

True


## Key terms

| Term                                | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| ----------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Token**                           | A sub-word piece of text. For example, "hello, world!" could be split into ["hello", ",", "world", "!"]. A token can be a whole word,<br> part of a word or group of punctuation characters. 1 token ~= 4 characters in English, 100 tokens ~= 75 words.<br> Text gets broken into tokens before being passed to an LLM.                                                                                                                                                                                                                                                                                  |
| **Embedding**                       | A learned numerical representation of a piece of data. For example, a sentence of text could be represented by a vector with<br> 768 values. Similar pieces of text (in meaning) will ideally have similar values.                                                                                                                                                                                                                                                                                                                                                                                        |
| **Embedding model**                 | A model designed to accept input data and output a numerical representation. For example, a text embedding model may take in 384 <br>tokens of text and turn it into a vector of size 768. An embedding model can and often is different to an LLM model.                                                                                                                                                                                                                                                                                                                                                 |
| **Similarity search/vector search** | Similarity search/vector search aims to find two vectors which are close together in high-demensional space. For example, <br>two pieces of similar text passed through an embedding model should have a high similarity score, whereas two pieces of text about<br> different topics will have a lower similarity score. Common similarity score measures are dot product and cosine similarity.                                                                                                                                                                                                         |
| **Large Language Model (LLM)**      | A model which has been trained to numerically represent the patterns in text. A generative LLM will continue a sequence when given a sequence. <br>For example, given a sequence of the text "hello, world!", a genertive LLM may produce "we're going to build a RAG pipeline today!".<br> This generation will be highly dependant on the training data and prompt.                                                                                                                                                                                                                                     |
| **LLM context window**              | The number of tokens a LLM can accept as input. For example, as of March 2024, GPT-4 has a default context window of 32k tokens<br> (about 96 pages of text) but can go up to 128k if needed. A recent open-source LLM from Google, Gemma (March 2024) has a context<br> window of 8,192 tokens (about 24 pages of text). A higher context window means an LLM can accept more relevant information<br> to assist with a query. For example, in a RAG pipeline, if a model has a larger context window, it can accept more reference items<br> from the retrieval system to aid with its generation.      |
| **Prompt**                          | A common term for describing the input to a generative LLM. The idea of "[prompt engineering](https://en.wikipedia.org/wiki/Prompt_engineering)" is to structure a text-based<br> (or potentially image-based as well) input to a generative LLM in a specific way so that the generated output is ideal. This technique is<br> possible because of a LLMs capacity for in-context learning, as in, it is able to use its representation of language to breakdown <br>the prompt and recognize what a suitable output may be (note: the output of LLMs is probable, so terms like "may output" are used). |


## What we're going to build

We're going to build RAG pipeline which enables us to chat with a PDF document, specifically an open-source [nutrition textbook](https://pressbooks.oer.hawaii.edu/humannutrition2/), ~1200 pages long.

You could call our project NutriChat!

We'll write the code to:

1. Open a PDF document (you could use almost any PDF here).
2. Format the text of the PDF textbook ready for an embedding model (this process is known as text splitting/chunking).
3. Embed all of the chunks of text in the textbook and turn them into numerical representation which we can store for later.
4. Build a retrieval system that uses vector search to find relevant chunks of text based on a query.
5. Create a prompt that incorporates the retrieved pieces of text.
6. Generate an answer to a query based on passages from the textbook.

The above steps can broken down into two major sections:

1. Document preprocessing/embedding creation (steps 1-3).
2. Search and answer (steps 4-6).

And that's the structure we'll follow.

It's similar to the workflow outlined on the NVIDIA blog which [details a local RAG pipeline](https://developer.nvidia.com/blog/rag-101-demystifying-retrieval-augmented-generation-pipelines/).

<img src="https://github.com/mrdbourke/simple-local-rag/blob/main/images/simple-local-rag-workflow-flowchart.png?raw=true" alt="flowchart of a local RAG workflow" />


## 1. Document/Text Processing and Embedding Creation

Ingredients:

- PDF document of choice.
- Embedding model of choice.

Steps:

1. Import PDF document.
2. Process text for embedding (e.g. split into chunks of sentences).
3. Embed text chunks with embedding model.
4. Save embeddings to file for later use (embeddings will store on file for many years or until you lose your hard drive).


### Import PDF Document

This will work with many other kinds of documents.

However, we'll start with PDF since many people have PDFs.

But just keep in mind, text files, email chains, support documentation, articles and more can also work.

We're going to pretend we're nutrition students at the University of Hawai'i, reading through the open-source PDF textbook [_Human Nutrition: 2020 Edition_](https://pressbooks.oer.hawaii.edu/humannutrition2/).

There are several libraries to open PDFs with Python but I found that [PyMuPDF](https://github.com/pymupdf/pymupdf) works quite well in many cases.

First we'll download the PDF if it doesn't exist.


In [3]:
import os 
import requests

#Get pdf path
pdf_path = "human-nutrition-text.pdf"

#download pdf if it does not exist 

if not os.path.exists(pdf_path):
    print(f"[INFO] files doesn't exist , downloading...")

    # The URL of the PDF you want to download
    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    # The local filename to save the downloaded file
    filename = pdf_path

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open a file in binary write mode and save the content to it
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"The file has been downloaded and saved as {filename}")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
else:
    print(f"File {pdf_path} exists.")

File human-nutrition-text.pdf exists.


PDF acquired!

We can import the pages of our PDF to text by first defining the PDF path and then opening and reading it with PyMuPDF (`import fitz`).

We'll write a small helper function to preprocess the text as it gets read. Note that not all text will be read in the same so keep this in mind for when you prepare your text.

We'll save each page to a dictionary and then append that dictionary to a list for ease of use later.


In [4]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    """
    Performs minor formatting on texts.
    """
    cleaned_text = text.replace('\n', " " ).strip()

    return cleaned_text

def open_and_read_pdf(pdf_path : str)-> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_text = []
    for page_number , page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text = text)
        pages_and_text.append({
            "page_number": page_number -41,
            "page_char_count": len(text),
            "page_word_count": len(text.split(" ")),
            "page_sentence_count_row": len(text.split(". ")),
            "page_token_count": len(text)/4,
            "text":text
            })
    return pages_and_text

pages_and_text = open_and_read_pdf(pdf_path = pdf_path)
pages_and_text[:2]

/Users/swayamsingal/miniconda3/envs/research_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

208it [00:01, 702.13it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_row': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_row': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [5]:
import random 

random.sample(pages_and_text , k=3)

[{'page_number': 524,
  'page_char_count': 1944,
  'page_word_count': 321,
  'page_sentence_count_row': 16,
  'page_token_count': 486.0,
  'text': 'The common occurrence of advanced xerophthalmia in children  who died from infectious diseases led scientists to hypothesize that  supplementing vitamin A in the diet for children with xerophthalmia  might reduce disease-related mortality. In Asia in the late 1980s,  targeted populations of children were administered vitamin A  supplements, and the death rates from measles and diarrhea  declined by up to 50 percent. Vitamin A supplementation in these  deficient populations did not reduce the number of children who  contracted these diseases, but it did decrease the severity of the  diseases so that they were no longer fatal. Soon after the results  of these studies were communicated to the rest of the world, the  World Health Organization (WHO) and the United Nations  Children’s Fund (UNICEF) commenced worldwide campaigns  against vitamin A

In [6]:
import pandas as pd

df= pd.DataFrame(pages_and_text)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_row,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,147,3,199.25,Contents Preface University of Hawai‘i at Mā...


### Get some stats on the text

Let's perform a rough exploratory data analysis (EDA) to get an idea of the size of the texts (e.g. character counts, word counts etc) we're working with.

The different sizes of texts will be a good indicator into how we should split our texts.

Many embedding models have limits on the size of texts they can ingest, for example, the [`sentence-transformers`](https://www.sbert.net/docs/pretrained_models.html) model [`all-mpnet-base-v2`](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) has an input size of 384 tokens.

This means that the model has been trained in ingest and turn into embeddings texts with 384 tokens (1 token ~= 4 characters ~= 0.75 words).

Texts over 384 tokens which are encoded by this model will be auotmatically reduced to 384 tokens in length, potentially losing some information.

We'll discuss this more in the embedding section.

For now, let's turn our list of dictionaries into a DataFrame and explore it.


In [7]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_row,page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,199.50,10.52,287.00
std,348.86,560.38,95.83,6.55,140.10
min,-41.00,0.00,1.00,1.00,0.00
25%,260.75,762.00,134.00,5.00,190.50
50%,562.50,1231.50,216.00,10.00,307.88
75%,864.25,1603.50,272.00,15.00,400.88
max,1166.00,2308.00,430.00,39.00,577.00


Okay, looks like our average token count per page is 287.

For this particular use case, it means we could embed an average whole page with the `all-mpnet-base-v2` model (this model has an input capacity of 384).


### Further text processing (splitting pages into sentences)

The ideal way of processing text before embedding it is still an active area of research.

A simple method I've found helpful is to break the text into chunks of sentences.

As in, chunk a page of text into groups of 5, 7, 10 or more sentences (these values are not set in stone and can be explored).

But we want to follow the workflow of:

`Ingest text -> split it into groups/chunks -> embed the groups/chunks -> use the embeddings`

Some options for splitting text into sentences:

1. Split into sentences with simple rules (e.g. split on ". " with `text = text.split(". ")`, like we did above).
2. Split into sentences with a natural language processing (NLP) library such as [spaCy](https://spacy.io/) or [nltk](https://www.nltk.org/).

Why split into sentences?

- Easier to handle than larger pages of text (especially if pages are densely filled with text).
- Can get specific and find out which group of sentences were used to help within a RAG pipeline.

> **Resource:** See [spaCy install instructions](https://spacy.io/usage).

Let's use spaCy to break our text into sentences since it's likely a bit more robust than just using `text.split(". ")`.


In [8]:
from spacy.lang.en import English

nlp = English()

# Add a sentencizer pipeline
nlp.add_pipe("sentencizer")

# Create document instance as an example 
doc = nlp("This is a sentence. This another sentence. I like elephants")
assert len(list(doc.sents)) == 3

list(doc.sents)

[This is a sentence., This another sentence., I like elephants]

In [9]:
for item in tqdm(pages_and_text):
    item["sentences"] = list(nlp(item["text"]).sents)
    
    #Make sure all sentences are strings
    
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    #count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])


00%|██████████████████████████████████████| 1208/1208 [00:01<00:00, 726.77it/s]

In [10]:
random.sample(pages_and_text , k=1)

[{'page_number': 388,
  'page_char_count': 1587,
  'page_word_count': 272,
  'page_sentence_count_row': 14,
  'page_token_count': 396.75,
  'text': 'Even a slight change in blood pH can affect body functions. Recall  that acidic conditions can cause protein denaturation, which stops  proteins from functioning. The body has several systems that hold  the blood pH within the normal range to prevent this from  happening. One of these is the circulating albumin. Albumin is  slightly acidic, and because it is negatively charged it balances the  many positively charged molecules, such as protons (H+), calcium,  potassium, and magnesium which are also circulating in the blood.  Albumin acts as a buffer against abrupt changes in the  concentrations of these molecules, thereby balancing blood pH and  maintaining the status quo. The protein hemoglobin also  participates in acid-base balance by binding and releasing protons.  Transport  Albumin and hemoglobin also play a role in molecular transpo

In [11]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_row,page_token_count,page_sentence_count_spacy
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,199.50,10.52,287.00,10.32
std,348.86,560.38,95.83,6.55,140.10,6.30
min,-41.00,0.00,1.00,1.00,0.00,0.00
25%,260.75,762.00,134.00,5.00,190.50,5.00
50%,562.50,1231.50,216.00,10.00,307.88,10.00
75%,864.25,1603.50,272.00,15.00,400.88,15.00
max,1166.00,2308.00,430.00,39.00,577.00,28.00


### Chunking our sentences together

Let's take a step to break down our list of sentences/text into smaller chunks.

As you might've guessed, this process is referred to as **chunking**.

Why do we do this?

1. Easier to manage similar sized chunks of text.
2. Don't overload the embedding models capacity for tokens (e.g. if an embedding model has a capacity of 384 tokens, there could be information loss if you try to embed a sequence of 400+ tokens).
3. Our LLM context window (the amount of tokens an LLM can take in) may be limited and requires compute power so we want to make sure we're using it as well as possible.

Something to note is that there are many different ways emerging for creating chunks of information/text.

For now, we're going to keep it simple and break our pages of sentences into groups of 10 (this number is arbitrary and can be changed, I just picked it because it seemed to line up well with our embedding model capacity of 384).

On average each of our pages has 10 sentences.

And an average total of 287 tokens per page.

So our groups of 10 sentences will also be ~287 tokens long.

This gives us plenty of room for the text to embedded by our `all-mpnet-base-v2` model (it has a capacity of 384 tokens).

To split our groups of sentences into chunks of 10 or less, let's create a function which accepts a list as input and recursively breaks into down into sublists of a specified size.


In [12]:
# define split size to turn groups of sentences into chunks

num_sentences_chunk_size = 10

#Create a function to split the list of text recursively into chunk size

def split_list(input_list :list,
               split_size:int = num_sentences_chunk_size) -> list[list[str]]:
    return [input_list[i:i+split_size] for i in range(0, len(input_list), split_size)]

test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [13]:
# loop through pages and texts and plit sentences into chunks

for item in tqdm(pages_and_text):
    item["sentence_chunks"] = split_list(input_list=item["sentences"])
    
    item["num_chunks"] = len(item["sentence_chunks"])


00%|███████████████████████████████████| 1208/1208 [00:00<00:00, 188929.79it/s]

In [14]:
random.sample(pages_and_text , k=1)

[{'page_number': 509,
  'page_char_count': 1815,
  'page_word_count': 284,
  'page_sentence_count_row': 20,
  'page_token_count': 453.75,
  'text': 'overweight children.11 The results of this study were that a higher  percentage of children who made the small changes maintained or  reduced their BMI in comparison to children of families given a  pedometer but not asked to also make physical activity or dietary  changes.12 Several more studies funded by the National Institutes of  Health and USDA are ongoing and are evaluating the effectiveness  of the “small-changes” approach in reducing weight gain.  In 2009, a report of the Joint Task Force of the American Society  for Nutrition, Institute of Food Technologists, and International  Food Information Council proposed that the “small-changes”  approach when supported at the community, industry, and  governmental levels will be more effective than current strategies in  gradually reducing the obesity rate in America.13  11. Rodearmel SJ, 

In [15]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_row,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,199.50,10.52,287.00,10.32,1.53
std,348.86,560.38,95.83,6.55,140.10,6.30,0.64
min,-41.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,260.75,762.00,134.00,5.00,190.50,5.00,1.00
50%,562.50,1231.50,216.00,10.00,307.88,10.00,1.00
75%,864.25,1603.50,272.00,15.00,400.88,15.00,2.00
max,1166.00,2308.00,430.00,39.00,577.00,28.00,3.00


### Splitting each chunk into its own item

We'd like to embed each chunk of sentences into its own numerical representation.

So to keep things clean, let's create a new list of dictionaries each containing a single chunk of sentences with relative information such as page number as well statistics about each chunk.


In [16]:
import re

#split each chunk into each item
pages_and_chunks =[]
for item in tqdm(pages_and_text):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict={}
        chunk_dict["page_number"] = item["page_number"]
        
        #join rge sentences together into a paragraph like structure
        joined_sentence_chunk = "".join (sentence_chunk).replace("  "," ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk
        
        #get some states on our chunks
        chunk_dict["chunk_char_count"] =len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk)/4
        
        pages_and_chunks.append(chunk_dict)
len(pages_and_chunks)
    


00%|████████████████████████████████████| 1208/1208 [00:00<00:00, 24538.31it/s]

1843

In [17]:
random.sample(pages_and_chunks , k=1)

[{'page_number': 822,
  'sentence_chunk': 'successful. Around the world, approximately 40 percent of infants are breastfed exclusively for the recommended 6 months.121314 New mothers must also pay careful consideration to their own nutritional requirements to help their bodies recover in the wake of the pregnancy. This is particularly true for women who breastfeed their babies, which increases the need in certain nutrients. Lactation Preparation for making breast milk, although begun in puberty, is not completed until a woman’s first pregnancy. Early in the first trimester, the cells that will secrete milk divide and multiply. Hormones play a major role in preparing the woman’s body to breastfeed, particularly during the second and third trimesters. At that point, levels of the hormone prolactin increase to stimulate the growth of the milk duct system, which initiates and maintains milk production. Also during pregnancy, progesterone stimulates growth of the alveoli, the clusters of ce

In [18]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,583.38,734.10,112.74,183.52
std,347.79,447.51,71.24,111.88
min,-41.00,12.00,3.00,3.00
25%,280.50,315.00,45.00,78.75
50%,586.00,745.00,115.00,186.25
75%,890.00,1118.00,173.00,279.50
max,1166.00,1830.00,297.00,457.50


page number increases to 1843 as there would be chunks belonging to same page as well , thus unique no of pages are still the same

Hmm looks like some of our chunks have quite a low token count.

How about we check for samples with less than 30 tokens (about the length of a sentence) and see if they are worth keeping?


In [19]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 16.75 | Text: PART XI CHAPTER 11. TRACE MINERALS Chapter 11. Trace Minerals | 649
Chunk token count: 6.5 | Text: Fat-Soluble Vitamins | 539
Chunk token count: 10.5 | Text: 442 | Health Consequences of Alcohol Abuse
Chunk token count: 5.25 | Text: Young Adulthood | 907
Chunk token count: 3.0 | Text: Iodine | 681


Looks like many of these are headers and footers of different pages.

They don't seem to offer too much information.

Let's filter our DataFrame/list of dictionaries to only include chunks with over 30 tokens in length.


In [20]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [21]:
random.sample(pages_and_chunks_over_min_token_len,k=1)

[{'page_number': 503,
  'sentence_chunk': 'high levels of activity (about one hour of exercise per day). Moreover, most members eat breakfast every day, watch fewer than ten hours of television per week, and weigh themselves at least once per week. About half of them lost weight on their own, and the other half used some type of weight-loss program. In most scientific studies successful weight loss is accomplished only by changing the diet and by increasing physical activity. Doing one without the other limits the amount of weight lost and the length of time that weight loss is sustained. On an individual level it is quite possible to achieve successful weight loss, as over ten thousand Americans can attest. Moreover, losing as little as 10 percent of your body weight can significantly improve health and reduce disease risk.4 You do not have to be overweight or obese to reap benefits from eating a healthier diet and increasing physical activity as both provide numerous benefits beyond 

### Embedding our text chunks

While humans understand text, machines understand numbers best.

An [embedding](https://vickiboykis.com/what_are_embeddings/index.html) is a broad concept.

But one of my favourite and simple definitions is "a useful numerical representation".

The most powerful thing about modern embeddings is that they are _learned_ representations.

Meaning rather than directly mapping words/tokens/characters to numbers directly (e.g. `{"a": 0, "b": 1, "c": 3...}`), the numerical representation of tokens is learned by going through large corpuses of text and figuring out how different tokens relate to each other.

Ideally, embeddings of text will mean that similar meaning texts have similar numerical representation.

> **Note:** Most modern NLP models deal with "tokens" which can be considered as multiple different sizes and combinations of words and characters rather than always whole words or single characters. For example, the string `"hello world!"` gets mapped to the token values `{15339: b'hello', 1917: b' world', 0: b'!'}` using [Byte pair encoding](https://en.wikipedia.org/wiki/Byte_pair_encoding) (or BPE via OpenAI's [`tiktoken`](https://github.com/openai/tiktoken) library). Google has a tokenization library called [SentencePiece](https://github.com/google/sentencepiece).

Our goal is to turn each of our chunks into a numerical representation (an embedding vector, where a vector is a sequence of numbers arranged in order).

Once our text samples are in embedding vectors, us humans will no longer be able to understand them.

However, we don't need to.

The embedding vectors are for our computers to understand.

We'll use our computers to find patterns in the embeddings and then we can use their text mappings to further our understanding.

Enough talking, how about we import a text embedding model and see what an embedding looks like.

To do so, we'll use the [`sentence-transformers`](https://www.sbert.net/docs/installation.html) library which contains many pre-trained embedding models.

Specifically, we'll get the `all-mpnet-base-v2` model (you can see the model's intended use on the [Hugging Face model card](https://huggingface.co/sentence-transformers/all-mpnet-base-v2#intended-uses)).


In [22]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path = "all-mpnet-base-v2",
                                      device="mps")

# Create a list of sentences to turn into numbers
sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]


# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

# See the embeddings
for sentence, embedding in embeddings_dict.items():
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: The Sentences Transformers library provides an easy and open-source way to create embeddings.
Embedding: [-2.07981411e-02  3.03164814e-02 -2.01218221e-02  6.86483532e-02
 -2.55255289e-02 -8.47687386e-03 -2.07035700e-04 -6.32377341e-02
  2.81606354e-02 -3.33353058e-02  3.02634630e-02  5.30720949e-02
 -5.03526367e-02  2.62288190e-02  3.33314389e-02 -4.51578423e-02
  3.63043919e-02 -1.37109228e-03 -1.20171141e-02  1.14946561e-02
  5.04510589e-02  4.70857024e-02  2.11912952e-02  5.14607430e-02
 -2.03745961e-02 -3.58889513e-02 -6.67914515e-04 -2.94393096e-02
  4.95859236e-02 -1.05639827e-02 -1.52013991e-02 -1.31756964e-03
  4.48196866e-02  1.56023065e-02  8.60380283e-07 -1.21387048e-03
 -2.37978902e-02 -9.09456110e-04  7.34487409e-03 -2.53933924e-03
  5.23370393e-02 -4.68042940e-02  1.66215282e-02  4.71578613e-02
 -4.15599458e-02  9.01952444e-04  3.60279121e-02  3.42215039e-02
  9.68226939e-02  5.94828613e-02 -1.64984949e-02 -3.51249315e-02
  5.92519483e-03 -7.07996951e-04 -2.4103

In [23]:
embeddings[0].shape

(768,)

In [24]:
embedding = embedding_model.encode("My favourite animal is the cow") 

In [25]:
embedding_model.to("mps")

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [26]:
%%time
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"]=embedding_model.encode(item["sentence_chunk"])

100%|███████████████████████████████████████| 1680/1680 [01:36<00:00, 17.48it/s]

CPU times: user 12min 27s, sys: 4min 27s, total: 16min 54s
Wall time: 1min 36s


In [28]:
%%time

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]
text_chunks[419]

CPU times: user 1.47 ms, sys: 24 μs, total: 1.49 ms
Wall time: 1.51 ms


'often. • Calm your “sweet tooth” by eating fruits, such as berries or an apple. • Replace sugary soft drinks with seltzer water, tea, or a small amount of 100 percent fruit juice added to water or soda water. The Food Industry: Functional Attributes of Carbohydrates and the Use of Sugar Substitutes In the food industry, both fast-releasing and slow-releasing carbohydrates are utilized to give foods a wide spectrum of functional attributes, including increased sweetness, viscosity, bulk, coating ability, solubility, consistency, texture, body, and browning capacity. The differences in chemical structure between the different carbohydrates confer their varied functional uses in foods. Starches, gums, and pectins are used as thickening agents in making jam, cakes, cookies, noodles, canned products, imitation cheeses, and a variety of other foods. Molecular gastronomists use slow- releasing carbohydrates, such as alginate, to give shape and texture to their fascinating food creations. Add

In [29]:
%%time 

#Embed all texts into batches
text_chunk_embedding  = embedding_model.encode(text_chunks , 
                                               batch_size =32 ,
                                               convert_to_tensors= True)

text_chunk_embedding

CPU times: user 1min 33s, sys: 30.5 s, total: 2min 4s
Wall time: 37.8 s


array([[ 0.06742427,  0.09022824, -0.00509553, ..., -0.02211548,
        -0.02321365,  0.01256903],
       [ 0.0552156 ,  0.05921397, -0.01661672, ..., -0.01204062,
        -0.01028468,  0.02273966],
       [ 0.02798016,  0.03398141, -0.02064266, ..., -0.00536188,
         0.02125601,  0.03130551],
       ...,
       [ 0.07705151,  0.00978557, -0.01218173, ..., -0.04086807,
        -0.07517633, -0.02405261],
       [ 0.10304512, -0.01647023,  0.00826849, ..., -0.05742176,
        -0.02828031, -0.02946864],
       [ 0.08637734, -0.0125359 , -0.01127469, ..., -0.05223794,
        -0.03367294, -0.02986607]], dtype=float32)

### Save embeddings to file

Since creating embeddings can be a timely process (not so much for our case but it can be for more larger datasets), let's turn our `pages_and_chunks_over_min_token_len` list of dictionaries into a DataFrame and save it.

In [30]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

### Chunking and embedding questions

> **Which embedding model should I use?**

This depends on many factors. My best advice is to experiment, experiment, experiment! 

If you want the model to run locally, you'll have to make sure it's feasible to run on your own hardware. 

A good place to see how different models perform on a wide range of embedding tasks is the [Hugging Face Massive Text Embedding Benchmark (MTEB) Leaderboard](https://huggingface.co/spaces/mteb/leaderboard).

> **What other forms of text chunking/splitting are there?**

There are a fair few options here too. We've kept it simple with groups of sentences.

For more, [Pinecone has a great guide on different kinds of chunking](https://www.pinecone.io/learn/chunking-strategies/) including for different kinds of data such as markdown and LaTeX.

Libraries such as [LangChain also have a good amount of in-built text splitting options](https://python.langchain.com/docs/modules/data_connection/document_transformers/).

> **What should I think about when creating my embeddings?**

Our model turns text inputs up to 384 tokens long in embedding vectors of size 768.

Generally, the larger the vector size, the more information that gets encoded into the embedding (however, this is not always the case, as smaller, better models can outperform larger ones).

Though with larger vector sizes comes larger storage and compute requirements.

Our model is also relatively small (420MB) in size compared to larger models that are available.

Larger models may result in better performance but will also require more compute.

So some things to think about:
* Size of input - If you need to embed longer sequences, choose a model with a larger input capacity.
* Size of embedding vector - Larger is generally a better representation but requires more compute/storage.
* Size of model - Larger models generally result in better embeddings but require more compute power/time to run.
* Open or closed - Open models allow you to run them on your own hardware whereas closed models can be easier to setup but require an API call to get embeddings.

> **Where should I store my embeddings?**

If you've got a relatively small dataset, for example, under 100,000 examples (this number is rough and only based on first hand experience), `np.array` or `torch.tensor` can work just fine as your dataset.

But if you've got a production system and want to work with 100,000+ embeddings, you may want to look into a [vector database]( https://en.wikipedia.org/wiki/Vector_database) (these have become very popular lately and there are many offerings).

### Document Ingestion and Embedding Creation Extensions

One major extension to the workflow above would to functionize it.

Or turn it into a script.

As in, take all the functionality we've created and package it into a single process (e.g. go from document -> embeddings file).

So you could input a document on one end and have embeddings come out the other end. The hardest part of this is knowing what kind of preprocessing your text may need before it's turned into embeddings. Cleaner text generally means better results.

